In [1]:
import os
import wget
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# from astroquery import sha
# from astropy import coordinates as coord
# from astropy import units as u
import requests
import pyvo as vo




/opt/homebrew/anaconda3/lib/python3.8/site-packages/astroquery/sha/__init__.py:14: UserWarning: Experimental: SHA has not yet been refactored to have its API match the rest of astroquery.
  warnings.warn("Experimental: SHA has not yet been refactored to have its "


In [2]:
#TODO: ESTABLISH DICTIONARY OF PLANET NAMES
#      CENTRALIZE TABLES

In [23]:
 systems_df = pd.read_csv('data/PS_2021.11.10_14.15.35.csv', sep=',', low_memory=False)
 transitspec_df = pd.read_csv('data/transitspec_2021.10.21_17.04.26.csv', sep=',', low_memory=False)
 emispec_df = pd.read_csv('data/emissionspec_2021.10.22_13.36.00.csv', sep=',', low_memory=False)

In [24]:
#what are the MOST ESSENTIAL features?
for col in systems_df.columns:
    print(col)

pl_name
hostname
default_flag
sy_snum
sy_pnum
pl_controv_flag
pl_orbper
pl_orbsmax
pl_rade
pl_radj
pl_bmasse
pl_bmassj
pl_bmassprov
pl_orbeccen
pl_insol
pl_eqt
ttv_flag
st_spectype
st_teff
st_rad
st_mass
st_met
st_metratio
st_logg
rastr
ra
decstr
dec
sy_dist
sy_vmag
sy_kmag
sy_gaiamag


In [27]:
names = systems_df['pl_name']
earth_masses = systems_df['pl_bmasse']
num_stars = systems_df['sy_snum']
num_planets = systems_df['sy_pnum']

#remove rows with nonzero 'contraversial' flag
systems_df = systems_df[systems_df['pl_controv_flag'] == 0]

#filter only for readings that are considered the 'default' reading for a particular planet
systems_defaultonly_df = systems_df[systems_df['default_flag'] == 1]

#the number of non-contraversial planets in the table
print(len(systems_defaultonly_df))

In [31]:
systems_defaultonly_df

,pl_name,hostname,default_flag,sy_snum,sy_pnum,pl_controv_flag,pl_orbper,pl_orbsmax,pl_rade,pl_radj,...,st_metratio,st_logg,rastr,ra,decstr,dec,sy_dist,sy_vmag,sy_kmag,sy_gaiamag
0,11 Com b,11 Com,1,2,1,0,326.030000,1.290000,NaN,NaN,...,[Fe/H],2.31,12h20m42.91s,185.178779,+17d47m35.71s,17.793252,93.1846,4.72307,2.282,4.44038
3,11 UMi b,11 UMi,1,1,1,0,516.219970,1.530000,NaN,NaN,...,[Fe/H],1.93,15h17m05.90s,229.274595,+71d49m26.19s,71.823943,125.3210,5.01300,1.939,4.56216
6,14 And b,14 And,1,1,1,0,185.840000,0.830000,NaN,NaN,...,[Fe/H],2.63,23h31m17.80s,352.824150,+39d14m09.01s,39.235837,75.4392,5.23133,2.331,4.91781
12,14 Her b,14 Her,1,1,2,0,1773.400020,2.930000,NaN,NaN,...,[Fe/H],4.45,16h10m24.50s,242.602101,+43d48m58.90s,43.816362,17.9323,6.61935,4.714,6.38300
15,16 Cyg B b,16 Cyg B,1,3,1,0,798.500000,1.660000,NaN,NaN,...,[Fe/H],4.36,19h41m51.75s,295.465642,+50d31m00.57s,50.516824,21.1397,6.21500,4.651,6.06428
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29962,tau Gem b,tau Gem,1,1,1,0,305.500000,1.170000,NaN,NaN,...,[Fe/H],1.96,07h11m08.33s,107.784723,+30d14m41.84s,30.244954,112.5370,4.37799,1.681,4.03996
29970,ups And b,ups And,1,2,3,0,4.617033,0.059222,NaN,NaN,...,NaN,NaN,01h36m47.60s,24.198353,+41d24m13.73s,41.403815,13.4054,4.09565,2.859,3.98687
29972,ups And c,ups And,1,2,3,0,241.258000,0.827774,NaN,NaN,...,NaN,NaN,01h36m47.60s,24.198353,+41d24m13.73s,41.403815,13.4054,4.09565,2.859,3.98687
29980,ups And d,ups And,1,2,3,0,1276.460000,2.513290,NaN,NaN,...,NaN,NaN,01h36m47.60s,24.198353,+41d24m13.73s,41.403815,13.4054,4.09565,2.859,3.98687
